In [1]:
import platform
print(platform.python_version())

3.13.1


In [2]:
import pint
ureg = pint.UnitRegistry()
Q_= ureg.Quantity

In [3]:
from dataclasses import fields


In [4]:
import toml

In [5]:
skip_fields = ['name',
 # 'value',
 # 'defined_symbol',
 # 'aliases',
 'is_position_set',
 'start_line',
 'start_col',
 'end_line',
 'end_col',
 'raw']

keep_fields = [

 'value',
 'defined_symbol',
 'aliases',
]
    

In [6]:
data = {
    '_prefixes':{},
    # '_suffixes':{},
    '_units':{},
    '_dimensions':{},
    # '_contexts':{},
    # 'system':{},
    # 'group':{},
}
def add_key_if_not_empty(dat, key, value):
    if value == () or value is None:
        return dat
    else:
        dat[key]=value
    return dat
    
def parse_simple_definition(definition, definition_type):
    attrs = [field.name for field in fields(definition) if field.name in keep_fields]
    dat = {}
    for attr in attrs:
        dat=add_key_if_not_empty(dat, attr, getattr(definition,attr))
    if definition_type =="_units" and hasattr(definition,'raw'):
        dat['value'] = definition.raw.split("=")[1].strip()
    if definition_type =="_dimensions" and hasattr(definition,'raw'):
        dat['value'] = definition.raw.split("=")[1].strip()
    if definition_type =="_prefixes" and hasattr(definition,'raw'):
        dat['value'] = definition.raw.split("=")[1].strip()
    return dat
prefixes = ureg._prefixes
for definition_type in data.keys():
    definitions = getattr(ureg, definition_type).values()
    for definition in definitions:
        if hasattr(definition,'raw'):
            data[definition_type][definition.name]=parse_simple_definition(definition, definition_type)

In [7]:
data['prefix'] = data.pop('_prefixes')
data['unit'] = data.pop('_units')
data['dimension'] = data.pop('_dimensions')

In [8]:
data['group']={}
for group in ureg._group_definitions:
    dat = {}
    for attr in ["using_group_names"]:
        dat=add_key_if_not_empty(dat, attr, getattr(group,attr))
    dat['definitions']={}
    for definition in group.definitions:
        dat['definitions'][definition.name]=parse_simple_definition(definition, '_units')
    data['group'][group.name] = dat

In [9]:
data['system']={}
for group in ureg._system_definitions:
    dat = {}
    for attr in ["using_group_names"]:
        dat=add_key_if_not_empty(dat, attr, getattr(group,attr))
    dat['rules']=[]
    for rule in group.rules:
        dat['rules'].append(rule.raw)
    data['system'][group.name] = dat

In [10]:
data['context']={}
for group in ureg._context_definitions:
    dat = {}
    for attr in ["aliases", "defaults", "redefinitions"]:
        dat=add_key_if_not_empty(dat, attr, getattr(group,attr))
    dat['relations']=[]
    for rule in group.relations:
        dat['relations'].append(rule.raw)
    data['system'][group.name] = dat

In [11]:
dat

{'defaults': {},
 'relations': ['[mass] / [length] <-> [length] / [mass]: 1 / value']}

In [12]:
# data['system']={}
# for group in ureg._system_definitions:
#     dat = {}
#     for attr in ["using_group_names"]:
#         dat=add_key_if_not_empty(dat, attr, getattr(group,attr))
#     dat['rules']=[]
#     for rule in group.rules:
#         d_={'new_unit_name':rule.new_unit_name}
#         if rule.old_unit_name:
#             d_['old_unit_name'] = rule.old_unit_name
#         dat['rules'].append(d_)
#     data['system'][group.name] = dat

In [13]:
with open('test.toml', 'w', encoding="utf-8") as f:
    toml.dump(data, f)

In [14]:
import tomli_w
with open('test.toml', 'wb') as f:
    tomli_w.dump(data, f)

In [15]:
with open('test.toml', 'r', encoding="utf-8") as f:
    config = toml.load(f)

In [16]:
config

{'prefix': {'quecto': {'value': '1e-30', 'defined_symbol': 'q'},
  'ronto': {'value': '1e-27', 'defined_symbol': 'r'},
  'yocto': {'value': '1e-24', 'defined_symbol': 'y'},
  'zepto': {'value': '1e-21', 'defined_symbol': 'z'},
  'atto': {'value': '1e-18', 'defined_symbol': 'a'},
  'femto': {'value': '1e-15', 'defined_symbol': 'f'},
  'pico': {'value': '1e-12', 'defined_symbol': 'p'},
  'nano': {'value': '1e-9', 'defined_symbol': 'n'},
  'micro': {'value': '1e-6',
   'defined_symbol': 'µ',
   'aliases': ['μ', 'u', 'mu', 'mc']},
  'milli': {'value': '1e-3', 'defined_symbol': 'm'},
  'centi': {'value': '1e-2', 'defined_symbol': 'c'},
  'deci': {'value': '1e-1', 'defined_symbol': 'd'},
  'deca': {'value': '1e+1', 'defined_symbol': 'da', 'aliases': ['deka']},
  'hecto': {'value': '1e2', 'defined_symbol': 'h'},
  'kilo': {'value': '1e3', 'defined_symbol': 'k'},
  'mega': {'value': '1e6', 'defined_symbol': 'M'},
  'giga': {'value': '1e9', 'defined_symbol': 'G'},
  'tera': {'value': '1e12', 'd

In [17]:
ureg_ = pint.UnitRegistry("test.toml")

KeyError: 'system'

In [ ]:
%debug

In [ ]:
import pint
ureg = pint.UnitRegistry()
Q_= ureg.Quantity

In [ ]:
ureg._group_definitions

In [ ]:
ureg._units['J']

In [ ]:
%debug

In [ ]:
q=1*ureg.degC
q.to("K")

In [ ]:
config['database']['level2'] = 'new added information'
with open('config.toml', 'w') as f:
    toml.dump(config, f)

In [ ]:
x = pint.definitions.Definition.from_string('degF = 9 / 5 * kelvin; offset: 255.372222')
x

In [ ]:
print(Q_(32.1, ureg.degF).to('degC'))
print(Q_(32.01, ureg.degF).to('degC'))
print(Q_(32.001, ureg.degF).to('degC'))
print(Q_(32.0001, ureg.degF).to('degC'))
print(Q_(32.00001, ureg.degF).to('degC'))
print(Q_(32.000001, ureg.degF).to('degC'))
print(Q_(32.0000001, ureg.degF).to('degC'))
print(Q_(32.00000001, ureg.degF).to('degC'))
print(Q_(32.000000001, ureg.degF).to('degC'))

In [ ]:
pint.__file__

In [ ]:
#  print(q(212, ureg.degF).to('degC'))
temps =[0,255.37,373.1339]
for temp in temps:
    q= Q_(temp,"K")
    print(q.to("K"), q.to("degC"), q.to("degF"),q.to("degR"))

In [ ]:
q

In [ ]:
print(q(32, ureg.degF).to('degK'))

In [ ]:
q(32, ureg.degF).to('degK').m


In [ ]:
assertTrue(abs(self.Q_(0, ureg.degC).to('degF').m- 32)<1e-8)

In [ ]:
import numpy as np
np.__version__

In [ ]:
import pint
print(pint.compat.__file__)

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
pint._HAS_PINTPANDAS

In [ ]:
ureg=pint.UnitRegistry()
Q_=ureg.Quantity

In [ ]:
a=Q_(1,"m")
b=a*Q_([[1,2,3]],"m")
c=Q_(3,"m")

In [ ]:
my_array = ureg.Quantity([1,2],"m")
my_array_mm = ureg.Quantity([1,2],"mm")
np.concatenate([my_array,my_array])

In [ ]:
getattr(np,"linspace")==np.linspace

In [ ]:
a

In [ ]:
np.power(a,[2, 3])

In [ ]:
from pint.testsuite.test_numpy import TestNumpyMathematicalFunctions
self = TestNumpyMathematicalFunctions()
self.ureg = ureg
self.Q_ = Q_

In [ ]:
np.unwrap([0,540]*self.ureg.deg)

In [ ]:
self.test_unwrap()

In [ ]:
self.assertQuantityEqual(np.swapaxes(self.q, 1,0), np.array([[1,2],[3,4]]).T * self.ureg.m)


In [ ]:
a = np.ones((3, 2))
q = self.Q_(a, "km")
self.assertEqual(q.u, np.compress((q==q[0,0]).any(0), q).u)

In [ ]:
np.sum([self.q]*2)

In [ ]:
self.assertQuantityEqual(np.atleast_1d(self.q), self.q)

In [ ]:
y = np.linspace(0 * ureg.meter, 0.1 * ureg.meter)
y

In [ ]:
a.tolist()

In [ ]:
ureg.meter._REGISTRY.Quantity

In [ ]:
np.sum([my_array,my_array_mm],axis=1)

In [ ]:
np.arange(a,c,a)

In [ ]:
def _():
#     import pdb
#     pdb.set_trace()
    np.arange(a,c)
_()

In [ ]:
%debug

In [ ]:
np.sum([my_array,my_array_mm],axis=1)

In [ ]:
def _([start, ]stop, [step, ]dtype=None):
    print(start,stop,step)

In [ ]:
Q_(np.array([1, 2]),"m").magnitude.dtype

In [ ]:
Q_(np.array([1, np.nan]),"m").magnitude.dtype

In [ ]:
import functools
import inspect
import numpy as np
import six
import sys


class ndarray(np.ndarray):
    """Updated version of numpy.ndarray."""
    def __array_function__(self, func, types, args, kwargs):
        # Cannot handle items that have __array_function__ other than our own.
        for t in types:
            if (hasattr(t, '__array_function__') and
                    t.__array_function__ is not ndarray.__array_function__):
                return NotImplemented

        # Arguments contain no overrides, so we can safely call the
        # overloaded function again.
        return func(*args, **kwargs)


def get_overloaded_types_and_args(relevant_args):
    """Returns a list of arguments on which to call __array_function__.
    
    __array_function__ implementations should be called in order on the return
    values from this function.
    """
    # Runtime is O(num_arguments * num_unique_types)
    overloaded_types = []
    overloaded_args = []
    for arg in relevant_args:
        arg_type = type(arg)
        if arg_type not in overloaded_types:
            try:
                array_function = arg_type.__array_function__
            except AttributeError:
                continue

            overloaded_types.append(arg_type)

            if array_function is not ndarray.__array_function__:
                index = len(overloaded_args)
                for i, old_arg in enumerate(overloaded_args):
                    if issubclass(arg_type, type(old_arg)):
                        index = i
                        break
                overloaded_args.insert(index, arg)

    return overloaded_types, overloaded_args


def full_name(obj):
    return f'{obj.__module__}.{obj.__qualname__}'
  

def attempt_augmented_error_message(error, append_message):
    """Attempt to recreate an error with an appended message."""
    try:
        return type(error)(error.args[0] + append_message, *error.args[1:])
    except Exception:
        return error
  
def try_array_function_override(func, relevant_arguments, args, kwargs):
    # TODO: consider simplifying the interface, to only require either `types`
    # (by calling __array_function__ a classmethod) or `overloaded_args` (by
    # dropping `types` from the signature of __array_function__)
    types, overloaded_args = get_overloaded_types_and_args(relevant_arguments)
    if not overloaded_args:
        return False, None

    for overloaded_arg in overloaded_args:
        # Note that we're only calling __array_function__ on the *first*
        # occurence of each argument type. This is necessary for reasonable
        # performance with a possibly long list of overloaded arguments, for
        # which each __array_function__ implementation might reasonably need to
        # check all argument types.
        try:
            result = overloaded_arg.__array_function__(
                func, types, args, kwargs)
        except Exception as error:
            # Ensure the type of the overloaded argument ends up in the
            # traceback
            message = (" [while calling {!r} implementation of {!r}]"
                       .format(full_name(type(overloaded_arg)),
                               full_name(func)))
            new_error = attempt_augmented_error_message(error, message)
            # Would probably need to use six to do this sanely on Python 2:
            # https://stackoverflow.com/questions/9157210/
            raise new_error.with_traceback(error.__traceback__) from None

        if result is not NotImplemented:
            return True, result

    raise TypeError('no implementation found for {} on types that implement '
                    '__array_function__: {}'
                    .format(func, list(map(type, overloaded_args))))


def array_function_dispatch(dispatcher):
    """Wrap a function for dispatch with the __array_function__ protocol."""
    def decorator(func):
        @functools.wraps(func)
        def new_func(*args, **kwargs):
            relevant_arguments = dispatcher(*args, **kwargs)
            success, value = try_array_function_override(
                new_func, relevant_arguments, args, kwargs)
            if success:
                return value
            return func(*args, **kwargs)
        return new_func
    return decorator


In [ ]:
import numpy as np


def _broadcast_to_dispatcher(array, shape, subok=None):
    return (array,)

@array_function_dispatch(_broadcast_to_dispatcher)
def broadcast_to(array, shape, subok=False):
    return np.broadcast_to(array, shape, subok)


def _concatenate_dispatcher(arrays, axis=None, out=None):
    for array in arrays:
        yield array
    if out is not None:
        yield out
    
@array_function_dispatch(_concatenate_dispatcher)
def concatenate(arrays, axis=0, out=None):
    return np.concatenate(arrays, axis=axis, out=out)

In [ ]:
HANDLED_FUNCTIONS = {}

class MyArray:
    def __array_function__(self, func, types, args, kwargs):
        if func not in HANDLED_FUNCTIONS:
            return NotImplemented
        if not all(issubclass(t, MyArray) for t in types):
            return NotImplemented
        return HANDLED_FUNCTIONS[func](*args, **kwargs)

def implements(numpy_function):
    """Register an __array_function__ implementation for MyArray objects."""
    def decorator(func):
        HANDLED_FUNCTIONS[numpy_function] = func
        return func
    return decorator

    
# dummy implementation to show how overloads work with new/unexpected arguments
@implements(concatenate)
def _(arrays):
    pass

my_array = MyArray()
concatenate([my_array])  # works
concatenate([my_array], axis=0)  # not supported

In [ ]:
HANDLED_FUNCTIONS = {}
    
def __array_function__(self, func, types, args, kwargs):
    if func not in HANDLED_FUNCTIONS:
        return NotImplemented
    if not all(issubclass(t, ureg.Quantity) for t in types):
        return NotImplemented
    return HANDLED_FUNCTIONS[func](*args, **kwargs)

def implements(numpy_function):
    """Register an __array_function__ implementation for MyArray objects."""
    def decorator(func):
        HANDLED_FUNCTIONS[numpy_function] = func
        return func
    return decorator

setattr(ureg.Quantity, '__array_function__', __array_function__)
    
# dummy implementation to show how overloads work with new/unexpected arguments
@implements(concatenate)
def _(arrays, axis=0, out=None):
    out_units = arrays[0].units
    arrays = [q.to(out_units) for q in arrays]
    return ureg.Quantity(np.concatenate(arrays, axis=axis, out=out), out_units)

my_array = ureg.Quantity([1,2],"m")
my_array_mm = ureg.Quantity([1,2],"mm")
concatenate([my_array,my_array_mm])  # works

In [ ]:
np.concatenate([my_array,my_array_mm])

In [ ]:
concatenate([my_array,my_array_mm], axis=0)  # not supported

In [ ]:
interpretations = '''
~ energy = work = enthalpy
    [energy] = [force] * [length]
    joule = newton * meter = J
    erg = dyne * centimeter
    btu = 1.05505585262e3 * joule = Btu = BTU = british_thermal_unit
    electron_volt = 1.60217653e-19 * J = eV
    quadrillion_btu = 10**15 * btu = quad
    thm = 100000 * BTU = therm = EC_therm
    calorie = 4.184 * joule = cal = thermochemical_calorie
    international_steam_table_calorie = 4.1868 * joule
    ton_TNT = 4.184e9 * joule = tTNT
    US_therm = 1.054804e8 * joule
    watt_hour = watt * hour = Wh = watthour
    hartree = 4.35974394e-18 * joule = = Eh = E_h = hartree_energy
    toe = 41.868e9 * joule = tonne_of_oil_equivalent
~ energy
~ torque
    [torque] = [force] * [length]
    newton_meter = newton * meter = Nm
~torque
'''

A single unit should return its interperetation only

A dual unit should check all valid units for its dimensionality

In [ ]:
nm=Q_(10.2345678,"Nm")
nmu=nm.units

nmu.dimensionality

In [ ]:
nmu._units

In [ ]:
n_m=Q_(10.2345678,"N m")
n_mu=n_m.units

n_mu.dimensionality

In [ ]:
n_mu._units

In [ ]:
pint.PintType._metadata

In [ ]:
from pintpandas.test_pandas_interface import *

In [ ]:
TestDataFrameAccessor().test_index_maintained()

In [ ]:
%debug

In [ ]:
scalars=PintArray([], dtype='pint[nanometer]')
next(i for i in scalars if hasattr(i,"units"))

In [ ]:
a=pint.PintType("m")

In [ ]:
self=pint.PintArray([1,2],"m")

In [ ]:
import numpy as np

In [ ]:
self.data

In [ ]:
pint.PintArray._from_sequence

In [ ]:
na_sentinel = -1
from pandas.core.algorithms import _factorize_array

arr, na_value = self._values_for_factorize()

labels, uniques = _factorize_array(arr, na_sentinel=na_sentinel,
                                   na_value=na_value)
uniques = self._from_factorized(uniques, self)


In [ ]:
labels, uniques

In [ ]:
import pandas as pd
a=pd.core.arrays.period_array([2000, 2001, 2002], freq='D')
type(a.dtype)

In [ ]:
type(a.astype(a.dtype))